In [1]:
#!pip install opencv-python
#!pip install tensorflow
#!pip install keras        Make sure we have all required packages
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
import os
import csv
from glob import glob
from tqdm import tqdm

In [ ]:
#Load video and convert to picture frames
count = 0
videoFile = "fish2.mp4"
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

In [ ]:
img = plt.imread('frame12.jpg')   # reading image using its name
plt.imshow(img)

In [ ]:
#bring in pre labeled images
df = pd.read_csv(r'C:\Users\gonza\Documents\Projects\fish.csv', low_memory = False,)
print(df)


In [ ]:
X = [ ]     # creating an empty array
for img_name in df.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)# storing each image in array X
    
X = np.array(X)    # converting list to array
X.shape

In [ ]:
y = df.Class
dummy_y = np_utils.to_categorical(y)    # one hot encoding Classes

In [ ]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)
X.shape

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import (
    VGG19, 
    preprocess_input, 
    decode_predictions
)
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)    # preparing the validation set

In [ ]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 

In [ ]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

In [ ]:
X_train = X_train.reshape(49, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(22, 7*7*512)
X_valid.shape

In [ ]:
train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()

In [ ]:
# i. Building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dense(4, activation='softmax'))    # output layer

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_valid.shape

In [ ]:
model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid))

In [ ]:
################# picture evaluate 
image_size = (224, 224)


In [ ]:
image_path = os.path.join('..', "Downloads", "fish.jpg")
img1 = image.load_img(image_path, target_size = image_size)
plt.imshow(img1)

In [ ]:
# Preprocess image for model prediction
# This step handles scaling and normalization for VGG19
x1 = image.img_to_array(img1)
x1 = np.expand_dims(x1, axis=0)
x1 = preprocess_input(x1)
x1.shape

In [ ]:
#predictions = model.predict(x1)

In [ ]:
count = 0
testvid = "fishv.mp4"
cap = cv2.VideoCapture(testvid)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

In [ ]:
test1 = pd.read_csv(r'C:\Users\gonza\Documents\Projects\fishtest.csv', low_memory = False,)
print(test1)


In [ ]:
X1 = [ ]     # creating an empty array
for img_name in test1.Image_ID:
    img = plt.imread('' + img_name)
    X1.append(img)# storing each image in array X
    
X1 = np.array(X1)    # converting list to array
X1.shape
test2 = []
for i in range(0,X1.shape[0]):
    a = resize(X1[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test2.append(a)
test2 = np.array(test2)

In [ ]:
#test

In [ ]:

# converting all the frames for a test video into numpy array
prediction_images = np.array(test)
# extracting features using pre-trained model
prediction_images = base_model.predict(prediction_images)
# converting features in one dimensional array
prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
prediction = model.predict_classes(prediction_images)

In [ ]:
X1.shape

In [ ]:
# preprocessing the images
#X1 = preprocess_input(X1)

# extracting features from the images using pretrained model
#X1 = base_model.predict(X1)

# converting the images to 1-D form
#X1 = X1.reshape(47, 224*224*3)

#X1 = X1/X1.max()
#predictions = np.argmax(model.predict(X))
X1.shape

In [ ]:
#predictions = model.predict_classes(X1)